In [68]:
from langchain_community.document_loaders import AmazonTextractPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.chains import LLMChain
import boto3
import botocore

config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 3}
)

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)

textract_client = boto3.client("textract", region_name="us-west-2")
textract_features=["LAYOUT"]
file_path = "s3://my-s3-doc-loader/Sample1.pdf"
loader = AmazonTextractPDFLoader(file_path,textract_features,client=textract_client)
docs = loader.load()

In [70]:
all_page_content = ""
for doc in docs:
    all_page_content += doc.page_content
print(all_page_content)


@PCPRO

PCPRO. CO. UK / NEWS
Briefing News

processors no small commitment after the unmitigated disaster of its previous effort to embrace ARM, Windows RT. However, it's now been suggested that the faults with the Surface hardware were nothing to do with Intel, but with Microsoft's own software drivers. This only came to light when Microsoft CEO Satya Nadella visited PC builder Lenovo and asked how the company was coping with theSkylake problems. According to Thurrott's sources, Lenovo's management were bewildered, telling Nadella they had experienced no reliability issues with Intel's processors at all. It appears Microsoft's CEO had been led up the garden path by his own staff. If true, it suggests Microsoft's engineers were so concerned about covering their backsides that they misled their own CEO. Microsoft declined to comment on the company's internal machinations.

## Consumer confidence

While the revelations might temporarily sour relations between Microsoft and Intel, the ro

In [72]:
template = """
Extract all the text datas in the given text documents and print all the text as a result in the formatted way as per the given document

Instruction:
- Extract all the datas ,Ensure not to skip any datas
- The given text document consists of two column,so you must give the result as the single column so that i can easy to be convert as a pdf
- Seperate the difference between the heading(title) and the paragraph 
- Strictly print the paragraph in the single line to convert the text in the document as single column
- Don't mention like title , paragraph , just print the title name instead of mentioning 'title'
- Some lines may miss the puntuations marks like "",'' so please add this if the marks has missed in the line


{datas}
"""
qa_prompt = PromptTemplate(template=template, input_variables=["datas"])
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 40000})
llm_chain = LLMChain(prompt=qa_prompt, llm=llm, verbose= False)
result = llm_chain.run(datas= all_page_content)

In [73]:
text_to_remove = "Here is the extracted text data from the given document formatted in a single column:"
cleaned_paragraph = result.replace(text_to_remove, "")

In [75]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

# Create a PDF document
doc = SimpleDocTemplate(r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\2-Cloumn page\output_2column\Output_sample1.pdf", pagesize=letter)

# Get the styles for the document
styles = getSampleStyleSheet()

# Create a list of paragraphs from the text
paragraphs = []
for line in cleaned_paragraph.split("\n"):
    paragraph = Paragraph(line, styles["BodyText"])
    paragraphs.append(paragraph)

# Build the PDF document
doc.build(paragraphs)